In [ ]:

# coding: utf-8

# In[36]:


from __future__ import print_function
from __future__ import division
import torch
import sys
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
from PIL import Image
from torch import autograd
from torch.autograd import Variable
import scipy.misc
from sklearn.metrics import confusion_matrix
from torch.utils.data import Dataset, DataLoader
from torchvision.utils import make_grid\
import pickle as pkl
print("PyTorch Version: ",torch.__version__)
print("Torchvision Version: ",torchvision.__version__)


# In[38]:


data_file = "shuffled_labels_bin_17.csv"
batch_size = 8
lr = 0.001
num_epochs = 5
num_output = 1


# In[3]:


cur_transform_tr = transforms.Compose([
transforms.Resize(256),
transforms.CenterCrop(224),
transforms.RandomHorizontalFlip(),
transforms.ToTensor(),
transforms.Normalize(
mean=[0.485, 0.456, 0.406],
std=[0.229, 0.224, 0.225]
)])


# In[4]:


cur_transform_val = transforms.Compose([
transforms.Resize(256),
transforms.CenterCrop(224),
transforms.ToTensor(),
transforms.Normalize(
mean=[0.485, 0.456, 0.406],
std=[0.229, 0.224, 0.225]
)])


# In[5]:


data_transforms = {}
data_transforms['train'] = cur_transform_tr
data_transforms['val'] = cur_transform_val
data_transforms['test'] = cur_transform_val


# In[6]:


image_root = "Interior_Images/"


# In[7]:


def load_next_image(image_path):
    image_path=os.path.join(image_root,image_path)
    img = Image.open(image_path)
    img = img.convert('RGB')
    return img


# In[8]:


class Nerlp_Images(Dataset):
    def __init__(self, filename,transform, mode='train'):
        self.transform = transform
        nerlp = pd.read_csv(filename, index_col=None)
#         self.features = ['Mattress','Pressure_cooker','Chair','Bed','Table','Clock','Electric_fan','Radio','Sewing_machine','Mobile','Landline','Refrigerator','Air_con_or_Cooler','Washing_Machine','Internet','Computer','Television']
#         self.features = ['Wealth_Index']
#         nerlp=nerlp.sample(frac=1)
        self.features = [sys.argv[1]]
        self.nerlp = nerlp[self.features].values
#         mx_val=np.amax(self.nerlp,axis=0)
#         self.nerlp = np.divide(self.nerlp,mx_val)
        self.houseids = nerlp['uniq_id'].values
        length=len(self.houseids)
        if(mode=='train'):
            start=int(0*length)
            end=int(0.8*length)
        elif(mode=='val'):
            start=int(0.8*length)
            end=int(0.9*length)
        else:
            start=int(0.9*length)
            end=int(1*length)
        
        self.nerlp=self.nerlp[start:end]
        self.houseids=self.houseids[start:end]
            
        # print(type(self.houseids))
        # print(self.houseids.shape)
        # self.mode = mode

    def __len__(self):
        return self.nerlp.shape[0]

    def __getitem__(self, idx):

        y = self.nerlp[idx].astype('float64')
        house_code = self.houseids[idx]
        image_name=str(house_code)+"_HouseInterior_6.jpg"
        x = load_next_image(image_name)
        x = self.transform(x)
        # print(type(x))
        # print(y.shape)
        if(x.shape[0]>3):
            print("********",image_name,"**********",x.shape)
        x=torch.tensor(x)
        y=torch.tensor(y)
        # print(x.shape)
        # print(y.shape)
        return x, y


# In[9]:


dsets = {
    x: Nerlp_Images(filename=data_file, transform=data_transforms[x], mode=x)
    for x in ['train', 'val','test']
}


# In[21]:


# dtype=torch.FloatTensor
dtype = torch.LongTensor
dset_loaders = {
    x: torch.utils.data.DataLoader(dsets[x], batch_size=batch_size, shuffle=(x == 'train'),num_workers=4)
    for x in ['train', 'val','test']
}


# In[11]:


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


# In[11]:


def get_R2(y, y_):
#     y = y.data.cpu().numpy()
#     y_ = y_.data.cpu().numpy()
    y_mean = np.mean(y, axis=0)
    ss_pred = np.linalg.norm(y-y_, axis=0)
    ss_reg = np.linalg.norm(y-y_mean, axis=0)
    r_2 = ss_pred/ss_reg
    return 1-r_2


# In[12]:


features = [sys.argv[1]]
# features = ['Mattress','Pressure_cooker','Chair','Bed','Table','Clock','Electric_fan','Radio','Sewing_machine','Mobile','Landline','Refrigerator','Air_con_or_Cooler','Washing_Machine','Internet','Computer','Television']


# In[39]:


def train_model(model, dataloaders, criterion, optimizer, num_epochs=25, is_inception=False):
    since = time.time()

    val_acc_history = []

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = -1.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0
            run_y = np.zeros((0,),dtype=np.int32)
            run_y_ = np.zeros((0,),dtype=np.int32)
            # Iterate over data.
            cnt = 0
            for inputs, labels in dataloaders[phase]:
                cnt = cnt + 1
                labels = labels.type(dtype)
                labels = torch.squeeze(labels,dim=1)
                inputs = inputs.to(device)
                labels = labels.to(device)
                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    # Get model outputs and calculate loss
                    # Special case for inception because in training it has an auxiliary output. In train
                    #   mode we calculate the loss by summing the final output and the auxiliary output
                    #   but in testing we only consider the final output.
                    if is_inception and phase == 'train':
                        # From https://discuss.pytorch.org/t/how-to-optimize-inception-model-with-auxiliary-classifiers/7958
                        outputs, aux_outputs = model(inputs)
                        loss1 = criterion(outputs, labels)
                        loss2 = criterion(aux_outputs, labels)
                        loss = loss1 + 0.4*loss2
                    else:
                        outputs = model(inputs)
#                         print(outputs.data.cpu().numpy().shape)
                        loss = criterion(outputs, labels)

                    _, preds = torch.max(outputs, 1)
                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
                run_y = np.concatenate((run_y,labels.data.cpu().numpy()),axis=0)
                run_y_ = np.concatenate((run_y_,preds.data.cpu().numpy()),axis=0)
                
#                 run_epoch_loss = running_loss / (cnt*batch_size)
#                 run_epoch_R2 = get_R2(run_y,run_y_)

#                 print('{} rLoss: {:.4f} rR2_AHI: {:.4f} rR2_WI: {:.4f}'.format(phase, run_epoch_loss, run_epoch_R2[0],run_epoch_R2[1]))
        
        
            epoch_loss = running_loss / len(dataloaders[phase].dataset)
#             epoch_R2 = get_R2(run_y,run_y_)
#             epoch_acc = np.mean(epoch_R2)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)
#             print('{} Loss: {:.4f}'.format(phase, epoch_loss))
#             for i in range(len(features)):
#                 print('R2_'+features[i]+' : ',epoch_R2[i])
            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))
            print(confusion_matrix(run_y,run_y_))
            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
            if phase == 'val':
                val_acc_history.append(epoch_acc)

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model, val_acc_history


# In[30]:


model_ft = models.resnet18(pretrained=True)
for param in model_ft.parameters():
    param.requires_grad = False
num_ftrs = model_ft.fc.in_features
# output 2 because 2 classes
model_ft.fc = nn.Linear(num_ftrs,2)
input_size = 224


# In[31]:


print(model_ft)


# In[32]:


model_ft = model_ft.to(device)


# In[33]:


params_to_update = []
for name,param in model_ft.named_parameters():
    if param.requires_grad == True:
        params_to_update.append(param)


# In[34]:


# criterion = nn.MSELoss()
criterion = nn.CrossEntropyLoss()
optimizer_ft = optim.SGD(params=params_to_update, lr=lr, momentum=0.9)


# In[40]:


model_ft, hist = train_model(model_ft,dset_loaders, criterion, optimizer_ft, num_epochs=num_epochs)

pkl_dump = open(sys.argv[1]+'_cross_entropy.pkl','wb')
pkl.dump(model_ft,pkl_dump)

